
# Figure 1 Clustering and ADTs- ResCITE

## Libraries

In [1]:
library(dplyr)
library(Seurat)
library(patchwork)
library(H5weaver)
library(hise)
library(tidyverse)
library(SeuratObject)
library(SeuratDisk)
library(ggrepel)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t


Loading required package: data.table


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


Loading required package: Matrix

Loading required package: rhdf5


Attaching package: ‘H5weaver’


The following objects are masked from ‘package:rhdf5’:

    h5dump, h5ls


Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ readr     2.1.5
✔ ggplot2   3.4.3     ✔ stringr   1.5.1
✔ lubridate 1.9.3     ✔ 

## Reading H5 files

In [2]:
h5s <- list.files(
    path = '/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/01_Data/scrublet', 
    pattern = 'filtered_feature_bc_matrix.h5$', 
    full.names = TRUE, 
    recursive = TRUE
)


In [3]:
fully <- lapply(h5s, function(x){

    mtx <- Read10X_h5(x) # reading the matrix
    
    so <- CreateSeuratObject(mtx$`Gene Expression`)
    
    so[['ADT']] <- CreateAssayObject(mtx$`Antibody Capture`)
    
    so$orig.ident <- strsplit(x,'/')[[1]][9]
    so$orig.ident <- strsplit(so$orig.ident,'_')[[1]][1]
    so$type <- substr(so$orig.ident,0,2)

    return(so) 
    })

Genome matrix has multiple modalities, returning a list of matrices for this genome

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Genome matrix has multiple modalities, returning a list of matrices for this genome

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Genome matrix has multiple modalities, returning a list of matrices for this genome

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Genome matrix has multiple modalities, returning a list of matrices for this genome

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:

In [4]:
scrub <- list.files(
    # Place your analysis in a folder by number experiment, or whatever might work for you
    # It helps if this worksheet in in the same folder
    path = '/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/01_Data/scrublet',
    pattern = '.csv$', # change this to 'sample_raw_feature_bc_matrix.h5$' if you need the raw h5s
    full.names = TRUE, 
    recursive = TRUE
)


In [5]:
scrubs <- lapply(scrub,read.csv)

scrubs <- scrubs %>% lapply(function(x){
    rownames(x) <- x$X
    x$X <- NULL
    x$Barcodes <- rownames(x)
    return(x)
})

In [8]:
for (i in c(1:8)){
    
    fully[[i]][[]]$Barcodes <- rownames(fully[[i]][[]])
    
    fully[[i]][[]] <- left_join(fully[[i]][[]], scrubs[[i]], by = 'Barcodes') # , by='row.names', all=TRUE
}

In [11]:
fully <- Reduce(merge,fully) %>%
    subset(subset=Predicted_Doublet=='False') %>% 
    JoinLayers()


In [12]:
head(fully)

,orig.ident,nCount_RNA,nFeature_RNA,nCount_ADT,nFeature_ADT,type,Barcodes,Doublet_Score,Predicted_Doublet,Doublet_Score.x,Predicted_Doublet.x,Doublet_Score.y,Predicted_Doublet.y
,<chr>,<dbl>,<int>,<dbl>,<int>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>
AAACAAGCAATTGAACACTTTAGG-1,BL05111,13302,4388,13535,127,BL,AAACAAGCAATTGAACACTTTAGG-1,0.03723764,False,0.03723764,False,0.03723764,False
AAACAAGCACAAATACACTTTAGG-1,BL05111,6375,3124,10241,125,BL,AAACAAGCACAAATACACTTTAGG-1,0.08790072,False,0.08790072,False,0.08790072,False
AAACAAGCATACTTCAACTTTAGG-1,BL05111,625,431,144311,130,BL,AAACAAGCATACTTCAACTTTAGG-1,0.01258178,False,0.01258178,False,0.01258178,False
AAACCAATCACTATGTACTTTAGG-1,BL05111,6118,3041,17478,129,BL,AAACCAATCACTATGTACTTTAGG-1,0.09899889,False,0.09899889,False,0.09899889,False
AAACCAATCATGAGGTACTTTAGG-1,BL05111,22733,5900,14822,128,BL,AAACCAATCATGAGGTACTTTAGG-1,0.03723764,False,0.03723764,False,0.03723764,False
AAACCAATCCGCTATAACTTTAGG-1,BL05111,9181,3722,12917,124,BL,AAACCAATCCGCTATAACTTTAGG-1,0.09052632,False,0.09052632,False,0.09052632,False
AAACCAATCTGTGACCACTTTAGG-1,BL05111,7964,3618,19663,128,BL,AAACCAATCTGTGACCACTTTAGG-1,0.03614458,False,0.03614458,False,0.03614458,False
AAACCAGGTGACATTCACTTTAGG-1,BL05111,8798,3849,10989,125,BL,AAACCAGGTGACATTCACTTTAGG-1,0.03507611,False,0.03507611,False,0.03507611,False
AAACGGGCAAACTGGCACTTTAGG-1,BL05111,8204,3595,12555,121,BL,AAACGGGCAAACTGGCACTTTAGG-1,0.02644231,False,0.02644231,False,0.02644231,False


### Figure 1

In [13]:
fully$sample2 <- fully$orig.ident

fully$sample2 <- gsub('BL05111','CWB-1', fully$sample2)
fully$sample2 <- gsub('BL05113','CWB-2', fully$sample2)
fully$sample2 <- gsub('BL05401','drop', fully$sample2) # dropping Sample05401
fully$sample2 <- gsub('BL05402','CWB-3', fully$sample2)
fully$sample2 <- gsub('PB05111','Ficoll-1', fully$sample2)
fully$sample2 <- gsub('PB05113','Ficoll-2', fully$sample2)
fully$sample2 <- gsub('PB05401','drop', fully$sample2) # dropping Sample05401
fully$sample2 <- gsub('PB05402','Ficoll-3', fully$sample2)

fully <- subset(fully, subset=sample2!='drop')

fully$pres <- fully$type

fully$pres <- gsub('BL','CWB',fully$pres)
fully$pres <- gsub('PB','Ficoll',fully$pres)

fully$sample3 <- fully$orig.ident

fully$sample3 <- gsub('BL05111','Sample-1', fully$sample3)
fully$sample3 <- gsub('BL05113','Sample-2', fully$sample3)

fully$sample3 <- gsub('BL05402','Sample-3', fully$sample3)
fully$sample3 <- gsub('PB05111','Sample-1', fully$sample3)
fully$sample3 <- gsub('PB05113','Sample-2', fully$sample3)

fully$sample3 <- gsub('PB05402','Sample-3', fully$sample3)


In [14]:
yl_or <- '#FAC05E'
blu <- '#1593CD'
sage <- '#91C7B1'
red <- '#B33951'
pal4 <- c(yl_or,blu,sage,red)
pal2 <- c(blu,red)

In [15]:
fully$pres <- factor(fully$pres, levels = c('Ficoll','CWB'))

In [16]:
fully[["percent.mt"]] <- PercentageFeatureSet(fully, pattern = "^MT-")
fully <- subset(fully, subset=percent.mt < 5)

In [17]:
fully <- NormalizeData(fully) %>% 
    FindVariableFeatures() %>% 
    ScaleData() %>% 
    RunPCA() %>% 
    RunUMAP(dims = 1:20) %>% 
    FindNeighbors(dims = 1:20) %>% 
    FindClusters(resolution = 0.5)


Normalizing layer: counts

Finding variable features for layer counts

Centering and scaling data matrix

PC_ 1 
Positive:  NCF2, SPI1, SERPINA1, CST3, IFI30, CLEC7A, MNDA, EMILIN2, HCK, CD68 
	   PLXNB2, LILRB2, TNFAIP2, LRRC25, ZNF385A, TYMP, MEFV, SULF2, FGL2, MPEG1 
	   CYBB, CPVL, LYZ, TIMP2, CSTA, CSF3R, PILRA, LRP1, SLC7A7, KLF4 
Negative:  IL7R, CD247, LTB, LEF1, CD7, TRBC1, PYHIN1, TRAT1, SAMD3, CD28 
	   PCED1B, CD69, CD27, CCR7, TRIB2, BCL2, ITGA6, MAL, IL2RB, TRABD2A 
	   KLRK1, CTSW, OBSCN, ITM2A, HIST1H1C, MYC, NELL2, KLRG1, PRF1, CST7 
PC_ 2 
Positive:  NIBAN3, BANK1, CD79A, IGHM, FCRL1, MS4A1, CD22, PAX5, IGHD, FCRL2 
	   TCL1A, TNFRSF13C, BCL11A, RUBCNL, POU2AF1, RALGPS2, FCER2, BLK, COBLL1, FCRLA 
	   AFF3, CD79B, BLNK, CXCR5, TSPAN13, IGKC, OSBPL10, WDFY4, COL19A1, ADAM28 
Negative:  NKG7, PRF1, CST7, KLRD1, GZMA, SRGN, CCL5, CTSW, SYNE1, ADGRG1 
	   SAMD3, S100A4, MYO1F, TBX21, IL2RB, GNLY, FGFBP2, ANXA1, TGFBR3, CX3CR1 
	   GZMH, C1orf21, MYBL1, KLRK1, FCRL6, PYHIN

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 42846
Number of edges: 1458900

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.9335
Number of communities: 21
Elapsed time: 17 seconds


## Celltype Label Transfer

In [18]:
ref <- LoadH5Seurat(file = '/home/jupyter/pbmc_multimodal.h5seurat')


Validating h5Seurat file

Initializing ADT with data

Adding counts for ADT

Adding variable feature information for ADT

Adding miscellaneous information for ADT

Initializing SCT with data

Adding counts for SCT

Adding variable feature information for SCT

Adding miscellaneous information for SCT

Adding reduction apca

Adding cell embeddings for apca

Adding feature loadings for apca

Adding miscellaneous information for apca

Adding reduction aumap

Adding cell embeddings for aumap

Adding miscellaneous information for aumap

Adding reduction pca

Adding cell embeddings for pca

Adding feature loadings for pca

Adding miscellaneous information for pca

Adding reduction spca

Adding cell embeddings for spca

Adding feature loadings for spca

Adding miscellaneous information for spca

Adding reduction umap

Adding cell embeddings for umap

Adding miscellaneous information for umap

Adding reduction wnn.umap

Adding cell embeddings for wnn.umap

Adding miscellaneous information for w

In [19]:
anchors <- FindTransferAnchors(
    reference = ref,
    query = fully,
    normalization.method = "SCT",
    reference.reduction = "spca",
    dims = 1:50
)


Normalizing query using reference SCT model

Projecting cell embeddings

Finding neighborhoods

Finding anchors

	Found 6230 anchors



In [20]:
fully <- TransferData(
    anchorset = anchors, 
    reference = ref, 
    query = fully,
    refdata = list(
        celltype.l1 = "celltype.l1",
        celltype.l2 = "celltype.l2",
        celltype.l3 = "celltype.l3",
        predicted_ADT = 'ADT'
    )
)


Finding integration vectors

Finding integration vector weights

Predicting cell labels

Warning message:
“Layer counts isn't present in the assay object; returning NULL”
Predicting cell labels

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Layer counts isn't present in the assay object; returning NULL”
Predicting cell labels

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Layer counts isn't present in the assay object; returning NULL”
Transfering 228 features onto reference data

Warning message:
“Layer counts isn't present in the assay object; returning NULL”


In [21]:
DefaultAssay(fully)<- 'ADT'

In [22]:
adt_cluster <- function(so){
    DefaultAssay(so) <- 'ADT'
    so <- so %>%
    NormalizeData(normalization.method='CLR',margin=2) %>%
    ScaleData() %>%
    RunPCA(features = rownames(so), reduction.name = "pca_adt", reduction.key = "pca_adt_", verbose = FALSE) %>% 
    RunUMAP(dims = 1:20, reduction='pca_adt', assay='ADT', reduction.name = 'adt.UMAP', reduction.key='adtUMAP_') %>% 
    FindNeighbors(dims = 1:20, graph.name = 'adt_snn') %>% 
    FindClusters(resolution = 0.5, 'adt_snn')
    return(so)
}

In [23]:
fully <- adt_cluster(fully)


Normalizing across cells

Centering and scaling data matrix

Warning message:
“Keys should be one or more alphanumeric characters followed by an underscore, setting key from pca_adt_ to pcaadt_”
18:04:36 UMAP embedding parameters a = 0.9922 b = 1.112

18:04:36 Read 42846 rows and found 20 numeric columns

18:04:36 Using Annoy for neighbor search, n_neighbors = 30

18:04:36 Building Annoy index with metric = cosine, n_trees = 50

0%   10   20   30   40   50   60   70   80   90   100%

[----|----|----|----|----|----|----|----|----|----|

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
|

18:04:40 Writing NN index file to temp file /tmp/RtmpWpIAwe/file10d90629e609b

18:04:40 Searching Annoy index using 1 thread, search_k = 3000

18:04:56 Annoy recall = 100%

18:04:58 Commencing smooth kNN distance calibration using 1 thread

18:04:59 Initializing from normalized Laplacian + noise

18:05:03 Commencing optimization for 200 epochs, with 186

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 42846
Number of edges: 400334

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.9179
Number of communities: 43
Elapsed time: 7 seconds


25 singletons identified. 18 final clusters.



In [24]:
saveRDS(fully,'/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/01_Data/Fig_1_Final.rds') # writing 


In [25]:
sessionInfo()

R version 4.1.3 (2022-03-10)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 20.04.4 LTS

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/openblas-pthread/libblas.so.3
LAPACK: /usr/lib/x86_64-linux-gnu/openblas-pthread/liblapack.so.3

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] ggrepel_0.9.5         SeuratDisk_0.0.0.9020 lubridate_1.9.3      
 [4] forcats_1.0.0         stringr_1.5.1         purrr_1.0.2          
 [7] readr_2.1.5           tidyr_1.3.1           tibble_3.2.1         
[10] ggplot2_3.4.3         tidyverse_2.0.0       hise_2.15.0          
[13] H5weaver_1.2.0        r

In [26]:
# 
